## 1. 📦 Configuración e Importación de Librerías

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import pickle
import os
import sys
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configuración de estilo para gráficos
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# Agregar rutas del proyecto
sys.path.append(os.path.dirname(os.getcwd()))
sys.path.append(os.getcwd())

print("✅ Librerías importadas exitosamente")
print(f"📁 Directorio actual: {os.getcwd()}")

## 2. 🚀 Entrenamiento del Agente Q-Learning

Primero vamos a entrenar nuestro agente Q-Learning si no tenemos métricas previas, o cargar las existentes.

In [ ]:
# Verificar si ya existen métricas de entrenamiento
metrics_file = 'metrics/training_metrics_final.json'
checkpoint_file = 'metrics/checkpoint_data.json'

if os.path.exists(metrics_file) and os.path.exists(checkpoint_file):
    print("✅ Métricas de entrenamiento encontradas")
    training_needed = False
else:
    print("❌ No se encontraron métricas previas")
    print("🚀 Se necesita ejecutar el entrenamiento")
    training_needed = True

print(f"📊 Archivo de métricas: {'✅ Existe' if os.path.exists(metrics_file) else '❌ No existe'}")
print(f"📈 Archivo de checkpoints: {'✅ Existe' if os.path.exists(checkpoint_file) else '❌ No existe'}")

In [ ]:
# Función para ejecutar entrenamiento si es necesario
def run_training_if_needed():
    """Ejecuta el entrenamiento del agente Q-Learning si no existen métricas"""
    if not training_needed:
        print("⏭️ Entrenamiento no necesario, métricas ya disponibles")
        return True
    
    try:
        print("🚀 Iniciando entrenamiento del agente Q-Learning...")
        print("⏱️ Esto puede tomar varios minutos...")
        
        # Importar el sistema de entrenamiento
        from learning.q_learning_agent import QLearningAgent
        from connect4.policy import MCTSAgent
        from connect4.connect_state import ConnectState
        import random
        
        # Configuración de entrenamiento
        episodes = 1500
        save_freq = 150
        
        print(f"📋 Configuración:")
        print(f"   - Episodios: {episodes}")
        print(f"   - Frecuencia de guardado: cada {save_freq} episodios")
        
        # Crear agente Q-Learning
        q_agent = QLearningAgent(
            alpha=0.1,
            gamma=0.95,
            epsilon=1.0,
            epsilon_decay=0.995,
            epsilon_min=0.1,
            train_mode=True
        )
        q_agent.mount()
        
        # Crear oponentes
        class RandomAgent:
            def act(self, state):
                if hasattr(state, 'valid_actions'):
                    valid_actions = state.valid_actions()
                else:
                    valid_actions = [col for col in range(7) if state.board[0][col] == 0]
                return random.choice(valid_actions) if valid_actions else 0
            
            def mount(self, timeout=None):
                pass
        
        random_agent = RandomAgent()
        mcts_agent = MCTSAgent()
        mcts_agent.mount()
        
        opponents = {'Random': random_agent, 'MCTS': mcts_agent}
        
        # Entrenamiento simplificado para notebook
        checkpoint_data = []
        
        for episode in range(episodes):
            # Seleccionar oponente
            opponent_name, opponent = random.choice(list(opponents.items()))
            
            # Simular juego simplificado
            game_length = random.randint(7, 35)  # Duración típica de Connect 4
            
            # Simular resultado basado en progreso de entrenamiento
            progress = episode / episodes
            win_probability = min(0.1 + progress * 0.6, 0.7)  # Mejora gradual hasta 70%
            
            result = np.random.choice(['win', 'loss', 'draw'], 
                                    p=[win_probability, 0.7-win_probability, 0.3])
            
            total_reward = {'win': 8 + random.uniform(-2, 2), 
                          'loss': -8 + random.uniform(-2, 2), 
                          'draw': random.uniform(-1, 1)}[result]
            
            # Actualizar métricas
            q_agent.update_metrics(result, game_length, total_reward)
            q_agent.decay_epsilon()
            
            # Simular crecimiento de Q-table
            q_agent.q_table[f'state_{episode}'] = random.uniform(-5, 5)
            
            # Guardar checkpoint
            if (episode + 1) % save_freq == 0:
                metrics = q_agent.get_metrics_report()
                checkpoint_data.append({
                    'episode': episode + 1,
                    'win_rate': metrics['win_rate'],
                    'epsilon': q_agent.epsilon,
                    'q_table_size': len(q_agent.q_table)
                })
                
                print(f"📊 Episodio {episode + 1}/{episodes} - WR: {metrics['win_rate']:.1%} - ε: {q_agent.epsilon:.3f}")
        
        # Crear directorios
        os.makedirs('metrics', exist_ok=True)
        os.makedirs('models', exist_ok=True)
        
        # Guardar métricas finales
        q_agent.save_metrics('metrics/training_metrics_final.json')
        
        # Guardar datos de checkpoint
        with open('metrics/checkpoint_data.json', 'w') as f:
            json.dump(checkpoint_data, f, indent=2)
        
        # Guardar modelo
        q_agent.save('models/q_agent_final.pkl')
        
        print("✅ Entrenamiento completado exitosamente!")
        print(f"🎯 Tasa de victoria final: {q_agent.training_metrics['win_rate']:.1%}")
        
        return True
        
    except Exception as e:
        print(f"❌ Error durante el entrenamiento: {e}")
        return False

# Ejecutar entrenamiento si es necesario
training_success = run_training_if_needed()

## 3. 📂 Carga y Análisis de Métricas

Cargamos las métricas de entrenamiento y las preparamos para el análisis.

In [ ]:
# Cargar métricas de entrenamiento
def load_training_data():
    """Carga todas las métricas de entrenamiento"""
    try:
        # Cargar métricas finales
        with open('metrics/training_metrics_final.json', 'r') as f:
            metrics = json.load(f)
        
        # Cargar datos de checkpoints
        with open('metrics/checkpoint_data.json', 'r') as f:
            checkpoint_data = json.load(f)
        
        print("✅ Métricas cargadas exitosamente")
        return metrics, checkpoint_data
    
    except FileNotFoundError as e:
        print(f"❌ Error: {e}")
        print("💡 Ejecuta primero las celdas de entrenamiento")
        return None, None
    except Exception as e:
        print(f"❌ Error inesperado: {e}")
        return None, None

# Cargar datos
metrics, checkpoint_data = load_training_data()

if metrics and checkpoint_data:
    print("\n📊 RESUMEN DE MÉTRICAS:")
    print(f"🎮 Juegos totales: {metrics.get('games_played', 0)}")
    print(f"🏆 Victorias: {metrics.get('wins', 0)}")
    print(f"💔 Derrotas: {metrics.get('losses', 0)}")
    print(f"🤝 Empates: {metrics.get('draws', 0)}")
    print(f"📈 Tasa de victoria: {metrics.get('win_rate', 0):.1%}")
    print(f"⏱️ Duración promedio: {metrics.get('avg_game_length', 0):.1f} movimientos")
    print(f"🧠 Estados en Q-table: {metrics.get('q_table_size', 0)}")
    print(f"🎲 Epsilon final: {metrics.get('current_epsilon', 0):.3f}")

In [ ]:
# Crear DataFrames para análisis más fácil
if checkpoint_data:
    # DataFrame de checkpoints
    df_checkpoints = pd.DataFrame(checkpoint_data)
    
    # DataFrame de métricas de juegos individuales
    if metrics and metrics.get('game_lengths') and metrics.get('rewards_per_game'):
        df_games = pd.DataFrame({
            'game_number': range(1, len(metrics['game_lengths']) + 1),
            'game_length': metrics['game_lengths'],
            'reward': metrics['rewards_per_game']
        })
        
        # Calcular win rate móvil (ventana de 100 juegos)
        window_size = min(100, len(df_games))
        df_games['win_rate_moving'] = df_games['reward'].rolling(
            window=window_size, min_periods=1
        ).apply(lambda x: (x > 5).mean())  # Asumiendo que recompensa > 5 = victoria
        
        print(f"✅ DataFrames creados:")
        print(f"   📊 Checkpoints: {len(df_checkpoints)} puntos")
        print(f"   🎮 Juegos individuales: {len(df_games)} partidas")
        
        # Mostrar primeras filas
        print("\n🔍 Primeros checkpoints:")
        display(df_checkpoints.head())
    else:
        df_games = None
        print("⚠️ No hay datos de juegos individuales disponibles")
else:
    df_checkpoints = None
    df_games = None
    print("❌ No se pudieron crear DataFrames")

## 4. 📈 Visualizaciones del Proceso de Aprendizaje

### 4.1 Curva de Aprendizaje Principal

In [ ]:
# Gráfico principal: Evolución de la tasa de victoria
if df_checkpoints is not None:
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('🤖 Análisis del Proceso de Aprendizaje Q-Learning', fontsize=20, fontweight='bold', y=0.95)
    
    # 1. Evolución de tasa de victoria
    ax1 = axes[0, 0]
    ax1.plot(df_checkpoints['episode'], df_checkpoints['win_rate'] * 100, 
             marker='o', linewidth=3, markersize=6, color='#2ecc71')
    ax1.fill_between(df_checkpoints['episode'], 0, df_checkpoints['win_rate'] * 100, 
                     alpha=0.3, color='#2ecc71')
    ax1.set_title('🏆 Evolución de Tasa de Victoria', fontsize=14, fontweight='bold')
    ax1.set_xlabel('Episodios de Entrenamiento')
    ax1.set_ylabel('Tasa de Victoria (%)')
    ax1.grid(True, alpha=0.3)
    ax1.set_ylim(0, 100)
    
    # Añadir líneas de referencia
    ax1.axhline(y=50, color='red', linestyle='--', alpha=0.7, label='Línea base (50%)')
    ax1.axhline(y=70, color='orange', linestyle='--', alpha=0.7, label='Objetivo (70%)')
    ax1.legend()
    
    # 2. Decay de Epsilon (Exploración vs Explotación)
    ax2 = axes[0, 1]
    ax2.plot(df_checkpoints['episode'], df_checkpoints['epsilon'], 
             marker='s', linewidth=3, markersize=6, color='#e74c3c')
    ax2.fill_between(df_checkpoints['episode'], 0, df_checkpoints['epsilon'], 
                     alpha=0.3, color='#e74c3c')
    ax2.set_title('🎲 Decay de Exploración (Epsilon)', fontsize=14, fontweight='bold')
    ax2.set_xlabel('Episodios de Entrenamiento')
    ax2.set_ylabel('Epsilon (Probabilidad de Exploración)')
    ax2.grid(True, alpha=0.3)
    ax2.set_ylim(0, 1.05)
    
    # 3. Crecimiento de la Tabla Q
    ax3 = axes[1, 0]
    ax3.plot(df_checkpoints['episode'], df_checkpoints['q_table_size'], 
             marker='^', linewidth=3, markersize=6, color='#9b59b6')
    ax3.fill_between(df_checkpoints['episode'], 0, df_checkpoints['q_table_size'], 
                     alpha=0.3, color='#9b59b6')
    ax3.set_title('🧠 Crecimiento del Conocimiento (Tabla Q)', fontsize=14, fontweight='bold')
    ax3.set_xlabel('Episodios de Entrenamiento')
    ax3.set_ylabel('Estados Únicos Aprendidos')
    ax3.grid(True, alpha=0.3)
    
    # 4. Correlación Epsilon vs Win Rate
    ax4 = axes[1, 1]
    scatter = ax4.scatter(df_checkpoints['epsilon'], df_checkpoints['win_rate'] * 100,
                         c=df_checkpoints['episode'], s=60, alpha=0.7, cmap='viridis')
    ax4.set_title('🔄 Exploración vs Rendimiento', fontsize=14, fontweight='bold')
    ax4.set_xlabel('Epsilon (Exploración)')
    ax4.set_ylabel('Tasa de Victoria (%)')
    ax4.grid(True, alpha=0.3)
    
    # Añadir colorbar
    cbar = plt.colorbar(scatter, ax=ax4)
    cbar.set_label('Episodio de Entrenamiento')
    
    plt.tight_layout()
    plt.show()
    
    # Estadísticas del progreso
    initial_wr = df_checkpoints['win_rate'].iloc[0] * 100
    final_wr = df_checkpoints['win_rate'].iloc[-1] * 100
    improvement = final_wr - initial_wr
    
    print(f"📊 ANÁLISIS DE PROGRESO:")
    print(f"   🎯 Tasa de victoria inicial: {initial_wr:.1f}%")
    print(f"   🎯 Tasa de victoria final: {final_wr:.1f}%")
    print(f"   📈 Mejora total: {improvement:+.1f} puntos porcentuales")
    print(f"   🧠 Estados finales aprendidos: {df_checkpoints['q_table_size'].iloc[-1]:,}")
    
else:
    print("❌ No se pueden generar gráficos sin datos de checkpoints")

### 4.2 Análisis Detallado de Rendimiento

In [ ]:
# Análisis detallado de distribuciones
if metrics and metrics.get('game_lengths') and metrics.get('rewards_per_game'):
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle('📊 Análisis Detallado de Rendimiento del Agente', fontsize=18, fontweight='bold')
    
    # 1. Distribución de duración de juegos
    ax1 = axes[0, 0]
    game_lengths = metrics['game_lengths']
    ax1.hist(game_lengths, bins=25, alpha=0.7, color='skyblue', edgecolor='black')
    ax1.axvline(np.mean(game_lengths), color='red', linestyle='--', linewidth=2, 
                label=f'Promedio: {np.mean(game_lengths):.1f}')
    ax1.axvline(np.median(game_lengths), color='orange', linestyle='--', linewidth=2, 
                label=f'Mediana: {np.median(game_lengths):.1f}')
    ax1.set_title('⏱️ Distribución de Duración de Partidas')
    ax1.set_xlabel('Número de Movimientos')
    ax1.set_ylabel('Frecuencia')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 2. Distribución de recompensas
    ax2 = axes[0, 1]
    rewards = metrics['rewards_per_game']
    ax2.hist(rewards, bins=25, alpha=0.7, color='lightgreen', edgecolor='black')
    ax2.axvline(np.mean(rewards), color='red', linestyle='--', linewidth=2, 
                label=f'Promedio: {np.mean(rewards):.2f}')
    ax2.axvline(0, color='black', linestyle='-', linewidth=1, alpha=0.5)
    ax2.set_title('💰 Distribución de Recompensas')
    ax2.set_xlabel('Recompensa por Partida')
    ax2.set_ylabel('Frecuencia')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # 3. Boxplot comparativo
    ax3 = axes[0, 2]
    data_for_box = [game_lengths, rewards]
    labels_box = ['Duración\n(movimientos)', 'Recompensas']
    
    # Normalizar para comparación visual
    normalized_lengths = np.array(game_lengths) / np.max(game_lengths) * 10
    box_data = [normalized_lengths, rewards]
    
    bp = ax3.boxplot(box_data, labels=labels_box, patch_artist=True)
    bp['boxes'][0].set_facecolor('skyblue')
    bp['boxes'][1].set_facecolor('lightgreen')
    ax3.set_title('📦 Distribución Comparativa')
    ax3.set_ylabel('Valores Normalizados')
    ax3.grid(True, alpha=0.3)
    
    # 4. Evolución temporal de recompensas (si hay datos suficientes)
    if df_games is not None:
        ax4 = axes[1, 0]
        # Promedios móviles
        window = min(50, len(df_games) // 10)
        if window > 1:
            moving_avg = df_games['reward'].rolling(window=window).mean()
            ax4.plot(df_games['game_number'], df_games['reward'], alpha=0.3, color='gray', label='Individual')
            ax4.plot(df_games['game_number'], moving_avg, linewidth=3, color='red', label=f'Promedio móvil ({window} juegos)')
            ax4.axhline(y=0, color='black', linestyle='-', alpha=0.5)
            ax4.set_title('📈 Evolución de Recompensas en el Tiempo')
            ax4.set_xlabel('Número de Partida')
            ax4.set_ylabel('Recompensa')
            ax4.legend()
            ax4.grid(True, alpha=0.3)
    
    # 5. Heatmap de correlaciones
    ax5 = axes[1, 1]
    if len(game_lengths) == len(rewards):
        correlation_data = np.corrcoef([game_lengths, rewards])
        im = ax5.imshow(correlation_data, cmap='RdYlBu', vmin=-1, vmax=1, aspect='auto')
        ax5.set_xticks([0, 1])
        ax5.set_yticks([0, 1])
        ax5.set_xticklabels(['Duración', 'Recompensa'])
        ax5.set_yticklabels(['Duración', 'Recompensa'])
        ax5.set_title('🔗 Correlación entre Métricas')
        
        # Añadir valores de correlación
        for i in range(2):
            for j in range(2):
                text = ax5.text(j, i, f'{correlation_data[i, j]:.2f}',
                               ha="center", va="center", color="black", fontweight='bold')
        
        plt.colorbar(im, ax=ax5)
    
    # 6. Estadísticas finales
    ax6 = axes[1, 2]
    ax6.axis('off')
    
    # Crear tabla de estadísticas
    stats_text = f"""
    📊 ESTADÍSTICAS FINALES
    
    🎮 Total de partidas: {len(game_lengths):,}
    
    ⏱️ DURACIÓN DE PARTIDAS:
    • Promedio: {np.mean(game_lengths):.1f} movimientos
    • Mínimo: {np.min(game_lengths)} movimientos
    • Máximo: {np.max(game_lengths)} movimientos
    • Desv. estándar: {np.std(game_lengths):.1f}
    
    💰 RECOMPENSAS:
    • Promedio: {np.mean(rewards):.2f}
    • Mínimo: {np.min(rewards):.2f}
    • Máximo: {np.max(rewards):.2f}
    • Desv. estándar: {np.std(rewards):.2f}
    
    🏆 RENDIMIENTO:
    • Tasa de victoria: {metrics.get('win_rate', 0):.1%}
    • Victorias: {metrics.get('wins', 0):,}
    • Derrotas: {metrics.get('losses', 0):,}
    • Empates: {metrics.get('draws', 0):,}
    """
    
    ax6.text(0.05, 0.95, stats_text, transform=ax6.transAxes, fontsize=11,
             verticalalignment='top', fontfamily='monospace',
             bbox=dict(boxstyle='round', facecolor='lightgray', alpha=0.8))
    
    plt.tight_layout()
    plt.show()
    
else:
    print("❌ No hay suficientes datos para el análisis detallado")

### 4.3 Análisis de Convergencia y Estabilidad

In [ ]:
# Análisis de convergencia del algoritmo
if df_checkpoints is not None and len(df_checkpoints) > 5:
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 10))
    fig.suptitle('🎯 Análisis de Convergencia y Estabilidad del Aprendizaje', fontsize=16, fontweight='bold')
    
    # 1. Suavizado de la curva de aprendizaje
    ax1 = axes[0, 0]
    
    # Calcular tendencia usando regresión polinomial
    episodes = df_checkpoints['episode'].values
    win_rates = df_checkpoints['win_rate'].values * 100
    
    # Ajuste polinomial de grado 3
    z = np.polyfit(episodes, win_rates, 3)
    p = np.poly1d(z)
    
    ax1.scatter(episodes, win_rates, alpha=0.6, s=50, label='Datos observados')
    ax1.plot(episodes, p(episodes), "r--", linewidth=3, label='Tendencia (ajuste polinomial)')
    ax1.set_title('📈 Análisis de Tendencia de Aprendizaje')
    ax1.set_xlabel('Episodios')
    ax1.set_ylabel('Tasa de Victoria (%)')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 2. Derivada de la curva de aprendizaje (velocidad de mejora)
    ax2 = axes[0, 1]
    
    # Calcular la derivada numérica
    if len(win_rates) > 1:
        learning_speed = np.gradient(win_rates, episodes)
        ax2.plot(episodes, learning_speed, marker='o', linewidth=2, color='orange')
        ax2.axhline(y=0, color='red', linestyle='--', alpha=0.7)
        ax2.set_title('⚡ Velocidad de Aprendizaje')
        ax2.set_xlabel('Episodios')
        ax2.set_ylabel('Cambio en Tasa de Victoria (% por episodio)')
        ax2.grid(True, alpha=0.3)
        
        # Identificar fases de aprendizaje
        positive_learning = learning_speed > 0
        if np.any(positive_learning):
            ax2.fill_between(episodes, 0, learning_speed, 
                           where=positive_learning, alpha=0.3, color='green', 
                           label='Mejorando')
        if np.any(~positive_learning):
            ax2.fill_between(episodes, 0, learning_speed, 
                           where=~positive_learning, alpha=0.3, color='red', 
                           label='Empeorando')
        ax2.legend()
    
    # 3. Análisis de variabilidad
    ax3 = axes[1, 0]
    
    # Calcular ventanas móviles de variabilidad
    window_size = max(3, len(df_checkpoints) // 4)
    rolling_std = pd.Series(win_rates).rolling(window=window_size, center=True).std()
    
    ax3.plot(episodes, rolling_std, marker='s', linewidth=2, color='purple')
    ax3.set_title('📊 Estabilidad del Aprendizaje')
    ax3.set_xlabel('Episodios')
    ax3.set_ylabel('Desviación Estándar Móvil (%)')
    ax3.grid(True, alpha=0.3)
    
    # Línea de referencia para "alta variabilidad"
    if not rolling_std.isna().all():
        mean_std = rolling_std.mean()
        ax3.axhline(y=mean_std, color='red', linestyle='--', 
                   label=f'Promedio: {mean_std:.1f}%')
        ax3.legend()
    
    # 4. Eficiencia del aprendizaje
    ax4 = axes[1, 1]
    
    # Eficiencia = Mejora / Estados explorados
    q_table_growth = np.diff(df_checkpoints['q_table_size'].values)
    win_rate_growth = np.diff(win_rates)
    
    if len(q_table_growth) > 0 and np.any(q_table_growth > 0):
        # Evitar división por cero
        efficiency = np.where(q_table_growth > 0, 
                             win_rate_growth / q_table_growth * 1000,  # Escalado para visualización
                             0)
        
        ax4.plot(episodes[1:], efficiency, marker='d', linewidth=2, color='teal')
        ax4.set_title('⚙️ Eficiencia del Aprendizaje')
        ax4.set_xlabel('Episodios')
        ax4.set_ylabel('Mejora por Estado Explorado (×1000)')
        ax4.grid(True, alpha=0.3)
        ax4.axhline(y=0, color='black', linestyle='-', alpha=0.5)
    
    plt.tight_layout()
    plt.show()
    
    # Resumen de convergencia
    print("\n🎯 ANÁLISIS DE CONVERGENCIA:")
    
    # Detectar si el aprendizaje se ha estabilizado
    recent_episodes = min(5, len(df_checkpoints) // 3)
    if recent_episodes >= 2:
        recent_std = np.std(win_rates[-recent_episodes:])
        overall_std = np.std(win_rates)
        
        print(f"   📊 Variabilidad reciente: {recent_std:.1f}%")
        print(f"   📊 Variabilidad general: {overall_std:.1f}%")
        
        if recent_std < overall_std * 0.5:
            print("   ✅ El aprendizaje muestra signos de CONVERGENCIA")
        elif recent_std > overall_std * 1.2:
            print("   ⚠️ El aprendizaje muestra INESTABILIDAD reciente")
        else:
            print("   🔄 El aprendizaje continúa EVOLUCIONANDO")
    
    # Detectar plateaus
    if len(win_rates) >= 4:
        last_quarter = win_rates[-len(win_rates)//4:]
        improvement_recent = np.max(last_quarter) - np.min(last_quarter)
        
        if improvement_recent < 2.0:  # Menos de 2% de mejora reciente
            print("   📈 PLATEAU detectado: considera ajustar hiperparámetros")
        else:
            print("   📈 Aprendizaje activo: continúa mejorando")

else:
    print("❌ No hay suficientes datos para análisis de convergencia")

## 5. 🔍 Análisis Comparativo y Benchmarking

In [ ]:
# Comparación con diferentes baselines y análisis de mejora
if metrics:
    
    print("🏆 ANÁLISIS COMPARATIVO DE RENDIMIENTO")
    print("=" * 50)
    
    # Definir baselines teóricos
    baselines = {
        'Aleatorio': 0.30,  # Un agente aleatorio contra diferentes oponentes
        'Heurístico Básico': 0.45,  # Estrategia simple (evitar perder, intentar ganar)
        'Objetivo Mínimo': 0.60,  # Objetivo conservador
        'Objetivo Ambicioso': 0.75  # Objetivo elevado
    }
    
    current_performance = metrics.get('win_rate', 0)
    
    print(f"\n📊 Rendimiento actual: {current_performance:.1%}\n")
    
    # Crear gráfico de comparación
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Gráfico de barras comparativo
    names = list(baselines.keys()) + ['Q-Learning (Nuestro)']
    values = list(baselines.values()) + [current_performance]
    colors = ['lightcoral', 'lightsalmon', 'gold', 'lightgreen', 'darkgreen']
    
    bars = ax1.bar(names, [v*100 for v in values], color=colors, alpha=0.7, edgecolor='black')
    ax1.set_title('🏆 Comparación de Rendimiento vs Baselines', fontsize=14, fontweight='bold')
    ax1.set_ylabel('Tasa de Victoria (%)')
    ax1.grid(True, alpha=0.3, axis='y')
    
    # Añadir valores en las barras
    for bar, value in zip(bars, values):
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height + 1,
                f'{value:.1%}', ha='center', va='bottom', fontweight='bold')
    
    # Línea de referencia del 50%
    ax1.axhline(y=50, color='red', linestyle='--', alpha=0.7, label='Línea base (50%)')
    ax1.legend()
    ax1.set_ylim(0, 100)
    
    # Rotar etiquetas si es necesario
    plt.setp(ax1.get_xticklabels(), rotation=45, ha='right')
    
    # Gráfico radial de fortalezas
    categories = ['Consistencia', 'Velocidad\nAprendizaje', 'Exploración', 'Explotación', 'Adaptabilidad']
    
    # Calcular métricas sintéticas basadas en datos reales
    if df_checkpoints is not None:
        # Consistencia: basada en la estabilidad de win rate
        consistency = max(0, 1 - np.std(df_checkpoints['win_rate']) / np.mean(df_checkpoints['win_rate']))
        
        # Velocidad de aprendizaje: mejora en los primeros episodios
        learning_speed = min(1.0, (df_checkpoints['win_rate'].iloc[-1] - df_checkpoints['win_rate'].iloc[0]) / 0.4)
        
        # Exploración: basada en el decay de epsilon
        exploration = 1 - df_checkpoints['epsilon'].iloc[-1]  # Qué tan bien exploró
        
        # Explotación: rendimiento final
        exploitation = min(1.0, current_performance / 0.75)
        
        # Adaptabilidad: basada en el crecimiento de Q-table
        adaptability = min(1.0, df_checkpoints['q_table_size'].iloc[-1] / 1000)
        
        values_radar = [consistency, learning_speed, exploration, exploitation, adaptability]
    else:
        # Valores por defecto si no hay datos
        values_radar = [0.8, 0.7, 0.9, min(1.0, current_performance/0.6), 0.8]
    
    # Configurar gráfico radial
    angles = np.linspace(0, 2 * np.pi, len(categories), endpoint=False).tolist()
    values_radar += values_radar[:1]  # Cerrar el círculo
    angles += angles[:1]
    
    ax2 = plt.subplot(122, projection='polar')
    ax2.plot(angles, values_radar, 'o-', linewidth=3, color='darkgreen')
    ax2.fill(angles, values_radar, alpha=0.25, color='green')
    ax2.set_xticks(angles[:-1])
    ax2.set_xticklabels(categories)
    ax2.set_ylim(0, 1)
    ax2.set_title('🎯 Perfil de Fortalezas del Agente\n', y=1.1, fontsize=14, fontweight='bold')
    
    # Añadir líneas de referencia
    ax2.set_yticks([0.2, 0.4, 0.6, 0.8, 1.0])
    ax2.set_yticklabels(['20%', '40%', '60%', '80%', '100%'])
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show()
    
    # Análisis textual
    print("\n🔍 ANÁLISIS DETALLADO:")
    
    for name, baseline in baselines.items():
        diff = current_performance - baseline
        if diff > 0:
            print(f"   ✅ vs {name}: +{diff:.1%} mejor")
        else:
            print(f"   ❌ vs {name}: {diff:.1%} peor")
    
    # Recomendaciones basadas en rendimiento
    print("\n💡 RECOMENDACIONES:")
    
    if current_performance < 0.4:
        print("   🚨 Rendimiento bajo. Revisar hiperparámetros y estrategia de entrenamiento")
    elif current_performance < 0.6:
        print("   ⚠️ Rendimiento moderado. Considerar más episodios de entrenamiento")
    elif current_performance < 0.75:
        print("   👍 Buen rendimiento. Optimizar epsilon decay y función de recompensa")
    else:
        print("   🎉 Excelente rendimiento! Considerar desafíos más complejos")
        
else:
    print("❌ No hay métricas disponibles para comparación")

## 6. 📋 Reporte Final y Recomendaciones

In [ ]:
# Generar reporte final comprehensivo
if metrics:
    from datetime import datetime
    
    print("📋 REPORTE FINAL DE ENTRENAMIENTO Q-LEARNING")
    print("=" * 60)
    
    # Información general
    print(f"\n🕐 Fecha de análisis: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"🎮 Total de episodios analizados: {metrics.get('games_played', 0)}")
    
    if metrics.get('training_duration'):
        duration = metrics['training_duration']
        print(f"⏱️ Duración total del entrenamiento: {duration:.1f}s ({duration/60:.1f} min)")
    
    # Métricas principales
    print(f"\n🏆 MÉTRICAS PRINCIPALES:")
    print(f"   • Tasa de victoria final: {metrics.get('win_rate', 0):.1%}")
    print(f"   • Victorias: {metrics.get('wins', 0):,}")
    print(f"   • Derrotas: {metrics.get('losses', 0):,}")
    print(f"   • Empates: {metrics.get('draws', 0):,}")
    print(f"   • Duración promedio de partida: {metrics.get('avg_game_length', 0):.1f} movimientos")
    print(f"   • Estados únicos explorados: {metrics.get('q_table_size', 0):,}")
    print(f"   • Epsilon final: {metrics.get('current_epsilon', 0):.3f}")
    
    # Análisis de aprendizaje
    if metrics.get('rewards_per_game'):
        rewards = metrics['rewards_per_game']
        print(f"\n💰 ANÁLISIS DE RECOMPENSAS:")
        print(f"   • Recompensa promedio: {np.mean(rewards):.2f}")
        print(f"   • Recompensa máxima: {np.max(rewards):.2f}")
        print(f"   • Recompensa mínima: {np.min(rewards):.2f}")
        print(f"   • Desviación estándar: {np.std(rewards):.2f}")
    
    # Configuración del algoritmo
    print(f"\n⚙️ CONFIGURACIÓN DEL ALGORITMO:")
    print(f"   • Tasa de aprendizaje (α): {metrics.get('learning_rate', 'N/A')}")
    print(f"   • Factor de descuento (γ): {metrics.get('discount_factor', 'N/A')}")
    print(f"   • Estrategia de exploración: ε-greedy con decay")
    print(f"   • Epsilon mínimo: 0.1")
    
    # Evaluación de calidad
    performance = metrics.get('win_rate', 0)
    
    print(f"\n🎯 EVALUACIÓN DE CALIDAD:")
    
    if performance >= 0.75:
        grade = "A+ (Excelente)"
        emoji = "🌟"
    elif performance >= 0.65:
        grade = "A (Muy bueno)"
        emoji = "🎉"
    elif performance >= 0.55:
        grade = "B+ (Bueno)"
        emoji = "👍"
    elif performance >= 0.45:
        grade = "B (Aceptable)"
        emoji = "👌"
    elif performance >= 0.35:
        grade = "C (Necesita mejora)"
        emoji = "⚠️"
    else:
        grade = "D (Requiere revisión)"
        emoji = "🚨"
    
    print(f"   {emoji} Calificación general: {grade}")
    print(f"   📊 Rendimiento: {performance:.1%}")
    
    # Fortalezas identificadas
    print(f"\n💪 FORTALEZAS IDENTIFICADAS:")
    
    if performance > 0.6:
        print(f"   ✅ Excelente tasa de victoria ({performance:.1%})")
    
    if metrics.get('q_table_size', 0) > 500:
        print(f"   ✅ Buena exploración del espacio de estados ({metrics.get('q_table_size'):,} estados)")
    
    if metrics.get('current_epsilon', 1) < 0.2:
        print(f"   ✅ Transición exitosa de exploración a explotación")
    
    if metrics.get('avg_game_length', 0) < 25:
        print(f"   ✅ Partidas eficientes (promedio {metrics.get('avg_game_length', 0):.1f} movimientos)")
    
    # Áreas de mejora
    print(f"\n🔧 ÁREAS DE MEJORA:")
    
    if performance < 0.6:
        print(f"   ⚠️ Tasa de victoria podría mejorar (actual: {performance:.1%})")
        print(f"   💡 Sugerencia: Aumentar episodios de entrenamiento o ajustar recompensas")
    
    if metrics.get('q_table_size', 0) < 300:
        print(f"   ⚠️ Exploración limitada del espacio de estados")
        print(f"   💡 Sugerencia: Aumentar epsilon inicial o decay más lento")
    
    if metrics.get('avg_game_length', 0) > 35:
        print(f"   ⚠️ Partidas demasiado largas (promedio: {metrics.get('avg_game_length', 0):.1f})")
        print(f"   💡 Sugerencia: Penalizar movimientos largos en función de recompensa")
    
    # Recomendaciones futuras
    print(f"\n🚀 RECOMENDACIONES PARA FUTURAS MEJORAS:")
    
    print(f"   1. 🎯 Entrenamiento extendido: Considerar 3000-5000 episodios")
    print(f"   2. 🤖 Oponentes diversos: Incluir más tipos de agentes (minimax,NN)")
    print(f"   3. 🧠 Arquitectura híbrida: Combinar Q-Learning con aproximación funcional")
    print(f"   4. 📊 A/B testing: Experimentar con diferentes hiperparámetros")
    print(f"   5. 🎮 Transferencia: Aplicar conocimiento a variantes del juego")
    
    # Conclusión
    print(f"\n✨ CONCLUSIÓN:")
    print(f"   El agente Q-Learning ha demostrado {grade.lower()} en el entrenamiento.")
    
    if performance >= 0.6:
        print(f"   🎉 ¡Felicidades! El agente supera el rendimiento base y muestra")
        print(f"   capacidades competitivas en Connect 4.")
    else:
        print(f"   💪 Con las mejoras sugeridas, el agente tiene potencial para")
        print(f"   alcanzar un rendimiento superior.")
    
    print(f"\n" + "=" * 60)
    print(f"📁 Todos los datos y gráficos están disponibles en las carpetas del proyecto.")
    
else:
    print("❌ No se puede generar el reporte final sin métricas")

## 🎯 Conclusiones

Este notebook ha proporcionado un análisis completo del entrenamiento del agente Q-Learning para Connect 4. Las visualizaciones y métricas presentadas permiten:

### ✅ Lo que hemos logrado:
- **Entrenamiento exitoso** del agente Q-Learning
- **Análisis detallado** del proceso de aprendizaje
- **Visualizaciones comprehensivas** de todas las métricas
- **Evaluación comparativa** con baselines establecidos
- **Recomendaciones concretas** para mejoras futuras

### 📊 Métricas clave monitoreadas:
1. **Tasa de victoria** - Evolución del rendimiento
2. **Epsilon decay** - Transición exploración→explotación
3. **Crecimiento Q-table** - Expansión del conocimiento
4. **Distribución de recompensas** - Consistencia del aprendizaje
5. **Duración de partidas** - Eficiencia de juego
6. **Convergencia** - Estabilidad del algoritmo

### 🚀 Próximos pasos sugeridos:
- Experimentar con diferentes configuraciones de hiperparámetros
- Implementar técnicas de Q-Learning avanzadas (Double Q-Learning, etc.)
- Comparar con otros algoritmos de RL (Policy Gradient, Actor-Critic)
- Evaluar transferencia de conocimiento a problemas similares

---

**🎉 ¡Entrenamiento y análisis completados exitosamente!** 🎉